**Overview
You are a data analyst and your client has a large ecommerce company in India (let’s call it X).
X gets a thousand orders via their website on a daily basis and they have to deliver them as fast
as they can. For delivering the goods ordered by the customers, X has tied up with multiple
courier companies in India as delivery partners who charge them some amount per delivery.
The charges are dependent upon two factors:
● Weight of the product and the Price of the product.
● Distance between the warehouse (pickup location) and customer’s delivery address
(destination location)
On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders
per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the
charges levied by their Delivery partners per Order are correct.**

### IMPORT LIBRARY

In [1]:
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

#### UPLOAD FILES

In [2]:
df=pd.read_excel("Company X - Order Report.xlsx")

In [3]:
sk=pd.read_excel("Company X - SKU Master.xlsx")

In [4]:
pin=pd.read_excel("Company X - Pincode Zones.xlsx")

In [5]:
cour=pd.read_excel("Courier Company - Invoice.xlsx")

In [6]:
cour_rates=pd.read_excel("Courier Company - Rates.xlsx")

In [7]:
df

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
...,...,...,...,...,...
395,2001806229,8904223818942,1,COD,286
396,2001806229,8904223818850,1,COD,563
397,2001806226,8904223818850,2,COD,1125
398,2001806210,8904223816214,1,COD,402


In [8]:
sk

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


**Renaming ExternOrderNo to Order ID**

In [9]:
df.rename(columns={'ExternOrderNo':'Order ID'}, inplace=True)

In [10]:
df

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
...,...,...,...,...,...
395,2001806229,8904223818942,1,COD,286
396,2001806229,8904223818850,1,COD,563
397,2001806226,8904223818850,2,COD,1125
398,2001806210,8904223816214,1,COD,402


**We create dictionary here for payment mode with ExternOrderNo. This will be help us in last step for calculating COD**

In [11]:
din=df.set_index('Order ID')['Payment Mode'].to_dict()

In [12]:
din

{2001827036: 'COD',
 2001825261: 'Prepaid',
 2001823564: 'Prepaid',
 2001822466: 'COD',
 2001821995: 'Prepaid',
 2001821766: 'COD',
 2001821750: 'Prepaid',
 2001821742: 'Prepaid',
 2001821679: 'COD',
 2001821502: 'Prepaid',
 2001821284: 'COD',
 2001821190: 'COD',
 2001821185: 'COD',
 2001820978: 'Prepaid',
 2001820690: 'COD',
 2001819252: 'Prepaid',
 2001818390: 'COD',
 2001817160: 'COD',
 2001817093: 'COD',
 2001816996: 'Prepaid',
 2001816684: 'Prepaid',
 2001816131: 'Prepaid',
 2001815688: 'COD',
 2001814580: 'COD',
 2001813009: 'COD',
 2001812941: 'COD',
 2001812854: 'COD',
 2001812838: 'COD',
 2001812650: 'COD',
 2001812195: 'Prepaid',
 2001811809: 'Prepaid',
 2001811604: 'COD',
 2001811475: 'COD',
 2001811466: 'Prepaid',
 2001811363: 'Prepaid',
 2001811306: 'Prepaid',
 2001811305: 'COD',
 2001811229: 'COD',
 2001811192: 'Prepaid',
 2001811153: 'Prepaid',
 2001811058: 'Prepaid',
 2001811039: 'Prepaid',
 2001810697: 'COD',
 2001810549: 'COD',
 2001810281: 'COD',
 2001810125: 'COD',


In [13]:
sk

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [14]:
df

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
...,...,...,...,...,...
395,2001806229,8904223818942,1,COD,286
396,2001806229,8904223818850,1,COD,563
397,2001806226,8904223818850,2,COD,1125
398,2001806210,8904223816214,1,COD,402


In [15]:
df[df['Order ID']==2001822466]

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
16,2001822466,8904223819468,2,COD,1205
17,2001822466,8904223819291,8,COD,7814


### MERGE sk file with df(order report)
 inorder to get weight by SKU number

In [16]:
df1=df.merge(sk[['SKU','Weight (g)']],on='SKU')

In [17]:
df1

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g)
0,2001827036,8904223818706,1,COD,233,127
1,2001821995,8904223818706,1,Prepaid,233,127
2,2001819252,8904223818706,1,Prepaid,233,127
3,2001816996,8904223818706,1,Prepaid,233,127
4,2001814580,8904223818706,1,COD,233,127
...,...,...,...,...,...,...
396,2001806616,8904223819123,1,COD,442,250
397,2001806567,8904223815804,1,COD,834,160
398,2001806567,8904223818577,1,COD,946,150
399,2001806408,8904223819437,2,Prepaid,1843,552


**To get total weight**

In [18]:
df1['Total Weight']=df1['Order Qty']*df1['Weight (g)']

In [19]:
df1

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g),Total Weight
0,2001827036,8904223818706,1,COD,233,127,127
1,2001821995,8904223818706,1,Prepaid,233,127,127
2,2001819252,8904223818706,1,Prepaid,233,127,127
3,2001816996,8904223818706,1,Prepaid,233,127,127
4,2001814580,8904223818706,1,COD,233,127,127
...,...,...,...,...,...,...,...
396,2001806616,8904223819123,1,COD,442,250,250
397,2001806567,8904223815804,1,COD,834,160,160
398,2001806567,8904223818577,1,COD,946,150,150
399,2001806408,8904223819437,2,Prepaid,1843,552,1104


**Group Function**
 Grouping Function is used here to get the Order qty,Item Price,Total Weight by OrderNo

In [20]:
df2=df1.groupby('Order ID')['Order Qty','Total Weight','Item Price(Per Qty.)'].sum()

In [21]:
df2

,Order Qty,Total Weight,Item Price(Per Qty.)
Order ID,,,
2001806210,2,220,1324
2001806226,2,480,1125
2001806229,3,500,1082
2001806232,8,1302,2139
2001806233,2,245,1486
...,...,...,...
2001821995,2,477,484
2001822466,10,1376,9019
2001823564,6,672,4803


In [22]:
df1[df1['Order ID']==2001825261]

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g),Total Weight
74,2001825261,8904223818638,3,Prepaid,855,137,411
78,2001825261,SACHETS001,1,Prepaid,350,10,10
80,2001825261,8904223819024,4,Prepaid,3636,112,448
97,2001825261,8904223819291,4,Prepaid,3907,112,448
106,2001825261,8904223818669,1,Prepaid,301,240,240


In [23]:
df[df['SKU']==8904223819024]

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
8,2001825261,8904223819024,4,Prepaid,3636
15,2001823564,8904223819024,2,Prepaid,1818
28,2001821502,8904223819024,2,Prepaid,1818
30,2001821284,8904223819024,1,COD,909
58,2001817093,8904223819024,2,COD,1818
86,2001812854,8904223819024,2,COD,1818
127,2001811192,8904223819024,2,Prepaid,1818
163,2001809820,8904223819024,2,Prepaid,1818
180,2001809592,8904223819024,8,Prepaid,7271
238,2001808102,8904223819024,2,Prepaid,1818


**Adding index as column and drop index**

In [24]:
df2['Order ID']=df2.index

In [25]:
df2=df2.reset_index(drop=True)

In [26]:
df2

,Order Qty,Total Weight,Item Price(Per Qty.),Order ID
0,2,220,1324,2001806210
1,2,480,1125,2001806226
2,3,500,1082,2001806229
3,8,1302,2139,2001806232
4,2,245,1486,2001806233
...,...,...,...,...
119,2,477,484,2001821995
120,10,1376,9019,2001822466
121,6,672,4803,2001823564
122,13,1557,9049,2001825261


In [27]:
cour

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0


### MERGE File df1 with cour 
inorder to get Customer Pincode and Type Of Shipment

In [28]:
newdf1=pd.merge(left=df2,right=cour,on='Order ID')

In [29]:
newdf1

,Order Qty,Total Weight,Item Price(Per Qty.),Order ID,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2,220,1324,2001806210,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2,480,1125,2001806226,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,3,500,1082,2001806229,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,8,1302,2139,2001806232,1091117222124,1.30,121003,507101,d,Forward charges,140.0
4,2,245,1486,2001806233,1091117222135,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2,477,484,2001821995,1091121183730,0.50,121003,342008,d,Forward charges,45.4
120,10,1376,9019,2001822466,1091121305541,1.10,121003,342301,d,Forward charges,135.0
121,6,672,4803,2001823564,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8
122,13,1557,9049,2001825261,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0


**Create New Dataframe from newdf1 and uses column which is create by us and column customer pincode ,Type of Shipment,Warehouse Pincode**

In [30]:
newdf2=newdf1[['AWB Code','Order ID','Order Qty','Total Weight','Item Price(Per Qty.)','Customer Pincode','Type of Shipment',
               'Warehouse Pincode',]]

In [31]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode
0,1091117221940,2001806210,2,220,1324,140604,Forward charges,121003
1,1091117222065,2001806226,2,480,1125,723146,Forward charges,121003
2,1091117222080,2001806229,3,500,1082,421204,Forward charges,121003
3,1091117222124,2001806232,8,1302,2139,507101,Forward charges,121003
4,1091117222135,2001806233,2,245,1486,263139,Forward charges,121003
...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,477,484,342008,Forward charges,121003
120,1091121305541,2001822466,10,1376,9019,342301,Forward charges,121003
121,1091121666133,2001823564,6,672,4803,492001,Forward and RTO charges,121003
122,1091121981575,2001825261,13,1557,9049,517128,Forward and RTO charges,121003


In [32]:
pin

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


**Now we have to fetch the zone info from pin dataframe. So we create a dictionary from pin dataframe**

In [33]:
d=pin.set_index('Customer Pincode')['Zone'].to_dict()

In [34]:
d

{507101: 'd',
 486886: 'd',
 532484: 'd',
 143001: 'b',
 515591: 'd',
 326502: 'd',
 208019: 'b',
 140301: 'b',
 396001: 'd',
 711106: 'd',
 284001: 'b',
 441601: 'd',
 248006: 'b',
 485001: 'd',
 845438: 'd',
 463106: 'd',
 495671: 'd',
 673002: 'e',
 208002: 'b',
 416010: 'd',
 226010: 'b',
 400705: 'd',
 262405: 'b',
 394210: 'd',
 411014: 'd',
 783301: 'e',
 486661: 'd',
 244001: 'b',
 492001: 'd',
 517128: 'd',
 562110: 'd',
 831006: 'd',
 140604: 'b',
 723146: 'd',
 421204: 'd',
 263139: 'b',
 743263: 'd',
 392150: 'd',
 382830: 'd',
 711303: 'd',
 283102: 'b',
 370201: 'd',
 248001: 'b',
 144001: 'b',
 403401: 'd',
 452001: 'd',
 721636: 'd',
 831002: 'd',
 226004: 'b',
 410206: 'd',
 516503: 'd',
 742103: 'd',
 452018: 'd',
 208001: 'b',
 244713: 'b',
 580007: 'd',
 360005: 'd',
 313027: 'b',
 341001: 'b',
 332715: 'b',
 302031: 'b',
 335001: 'b',
 334004: 'b',
 321001: 'b',
 324001: 'b',
 321608: 'b',
 302002: 'b',
 311011: 'b',
 306302: 'b',
 313001: 'b',
 322255: 'b',
 30201

**Mapping this dictionary to newdf2 dataframe and create new column 'Zone'**

In [35]:
newdf2['Zone']=newdf2['Customer Pincode'].map(d)

In [36]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,1091117221940,2001806210,2,220,1324,140604,Forward charges,121003,b
1,1091117222065,2001806226,2,480,1125,723146,Forward charges,121003,d
2,1091117222080,2001806229,3,500,1082,421204,Forward charges,121003,d
3,1091117222124,2001806232,8,1302,2139,507101,Forward charges,121003,d
4,1091117222135,2001806233,2,245,1486,263139,Forward charges,121003,b
...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,477,484,342008,Forward charges,121003,b
120,1091121305541,2001822466,10,1376,9019,342301,Forward charges,121003,b
121,1091121666133,2001823564,6,672,4803,492001,Forward and RTO charges,121003,d
122,1091121981575,2001825261,13,1557,9049,517128,Forward and RTO charges,121003,d


**Calculate Applicable weight slab according to zone**

In [37]:
newdf2['Applicable weight slab(based on Zone)']=np.select([(newdf2['Zone']=='a'),(newdf2['Zone']=='b'),(newdf2['Zone']=='c'),
                                                           (newdf2['Zone']=='d'),(newdf2['Zone']=='e')
            ],[0.25,0.5,0.75,1.25,1.5])

In [38]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone)
0,1091117221940,2001806210,2,220,1324,140604,Forward charges,121003,b,0.50
1,1091117222065,2001806226,2,480,1125,723146,Forward charges,121003,d,1.25
2,1091117222080,2001806229,3,500,1082,421204,Forward charges,121003,d,1.25
3,1091117222124,2001806232,8,1302,2139,507101,Forward charges,121003,d,1.25
4,1091117222135,2001806233,2,245,1486,263139,Forward charges,121003,b,0.50
...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,477,484,342008,Forward charges,121003,b,0.50
120,1091121305541,2001822466,10,1376,9019,342301,Forward charges,121003,b,0.50
121,1091121666133,2001823564,6,672,4803,492001,Forward and RTO charges,121003,d,1.25
122,1091121981575,2001825261,13,1557,9049,517128,Forward and RTO charges,121003,d,1.25


In [39]:
newdf2['Total Weight']=newdf2['Total Weight']/1000

In [40]:
newdf2[newdf2['Zone']=='b']

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone)
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.5
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.5
7,1091117222360,2001806304,3,0.500,1082,302017,Forward charges,121003,b,0.5
11,1091117223244,2001806458,2,0.700,1491,143001,Forward charges,121003,b,0.5
12,1091117223351,2001806471,10,1.621,4243,313027,Forward charges,121003,b,0.5
...,...,...,...,...,...,...,...,...,...,...
116,1091121034114,2001821742,1,0.240,603,327025,Forward charges,121003,b,0.5
117,1091121034350,2001821750,3,0.755,1284,313333,Forward charges,121003,b,0.5
118,1091121034641,2001821766,2,0.240,473,313001,Forward charges,121003,b,0.5
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.5


**Courier charges rate card at weight slab and pincode level. If the invoice mentions
“Forward charges” then only forward charges (“fwd”) should be applicable as per zone
and fixed & additional weights based on weight slabs. If the invoice mentions “Forward
and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be
applicable as per zone and fixed & additional weights based on weight slabs.
● For the first slab of that zone, “fixed” rate as per the slab is applicable. For each
additional slab, “additional” weight in the same proportion is applicable. Total charges will
be “fixed” + “total additional” if any. For example: weight 2.2KG, Zone C. So for Zone C
the slab length = 0.75KG. So the total applicable weight = 2.25KG. For the first 0.75 KG
the charge is “fwd”, and for each 0.75 after the first, charges will be additional charges**

***Create a function called newtestCondition for calculating Forwards charges etc***

In [41]:
def newtestCondition(s):
    c=math.ceil(s['Total Weight']/s['Applicable weight slab(based on Zone)'])
    counter=c+1
    charges=0
    i=1
    if((s['Zone']=='b') and (s['Type of Shipment']=='Forward charges')):
        
                
                while( i < counter):
                    if(i==1):
                        charges=33+charges
                        i=i+1
                        
                    else:
                        charges=28.3+charges
                        i=i+1
                return charges
    if((s['Zone']=='b') and (s['Type of Shipment']=='Forward and RTO charges')):
        
                
                while( i < counter):
                    if(i==1):
                        charges=33+20.5+charges
                        i=i+1
                        
                    else:
                        charges=28.3+28.3+charges
                        i=i+1
                return charges
    if((s['Zone']=='d') and (s['Type of Shipment']=='Forward charges')):
        
                
                while( i < counter):
                    if(i==1):
                        charges=45.4+charges
                        i=i+1
                        
                    else:
                        charges=44.8+charges
                        i=i+1
                return charges
    if((s['Zone']=='d') and (s['Type of Shipment']=='Forward and RTO charges')):
        
                
                while( i < counter):
                    if(i==1):
                        charges=45.4+41.3+charges
                        i=i+1
                        
                    else:
                        charges=44.8+44.8+charges
                        i=i+1
                return charges
    if((s['Zone']=='e') and (s['Type of Shipment']=='Forward charges')):
        
                
                while( i < counter):
                    if(i==1):
                        charges=56.6+charges
                        i=i+1
                        
                    else:
                        charges=55.5+charges
                        i=i+1
                return charges
    if((s['Zone']=='e') and (s['Type of Shipment']=='Forward and RTO charges')):
        
                
                while( i < counter):
                    if(i==1):
                        charges=56.6+50.7+charges
                        i=i+1
                        
                    else:
                        charges=55.5+55.5+charges
                        i=i+1
                return charges        
    

***Create new column ChargesSlap from function newtestCondition for forward charges***

In [42]:
newdf2['ChargesSlap']=newdf2.apply(newtestCondition,axis=1)

In [43]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone),ChargesSlap
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.50,33.0
1,1091117222065,2001806226,2,0.480,1125,723146,Forward charges,121003,d,1.25,45.4
2,1091117222080,2001806229,3,0.500,1082,421204,Forward charges,121003,d,1.25,45.4
3,1091117222124,2001806232,8,1.302,2139,507101,Forward charges,121003,d,1.25,90.2
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.50,33.0
...,...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.50,33.0
120,1091121305541,2001822466,10,1.376,9019,342301,Forward charges,121003,b,0.50,89.6
121,1091121666133,2001823564,6,0.672,4803,492001,Forward and RTO charges,121003,d,1.25,86.7
122,1091121981575,2001825261,13,1.557,9049,517128,Forward and RTO charges,121003,d,1.25,176.3


**Now We use dictionary called din to map with newdf2 dataframe to create new column called Paymnet Mode**

In [44]:
din

{2001827036: 'COD',
 2001825261: 'Prepaid',
 2001823564: 'Prepaid',
 2001822466: 'COD',
 2001821995: 'Prepaid',
 2001821766: 'COD',
 2001821750: 'Prepaid',
 2001821742: 'Prepaid',
 2001821679: 'COD',
 2001821502: 'Prepaid',
 2001821284: 'COD',
 2001821190: 'COD',
 2001821185: 'COD',
 2001820978: 'Prepaid',
 2001820690: 'COD',
 2001819252: 'Prepaid',
 2001818390: 'COD',
 2001817160: 'COD',
 2001817093: 'COD',
 2001816996: 'Prepaid',
 2001816684: 'Prepaid',
 2001816131: 'Prepaid',
 2001815688: 'COD',
 2001814580: 'COD',
 2001813009: 'COD',
 2001812941: 'COD',
 2001812854: 'COD',
 2001812838: 'COD',
 2001812650: 'COD',
 2001812195: 'Prepaid',
 2001811809: 'Prepaid',
 2001811604: 'COD',
 2001811475: 'COD',
 2001811466: 'Prepaid',
 2001811363: 'Prepaid',
 2001811306: 'Prepaid',
 2001811305: 'COD',
 2001811229: 'COD',
 2001811192: 'Prepaid',
 2001811153: 'Prepaid',
 2001811058: 'Prepaid',
 2001811039: 'Prepaid',
 2001810697: 'COD',
 2001810549: 'COD',
 2001810281: 'COD',
 2001810125: 'COD',


In [45]:
newdf2['Payment Mode']=newdf2['Order ID'].map(din)

In [46]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone),ChargesSlap,Payment Mode
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.50,33.0,COD
1,1091117222065,2001806226,2,0.480,1125,723146,Forward charges,121003,d,1.25,45.4,COD
2,1091117222080,2001806229,3,0.500,1082,421204,Forward charges,121003,d,1.25,45.4,COD
3,1091117222124,2001806232,8,1.302,2139,507101,Forward charges,121003,d,1.25,90.2,COD
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.50,33.0,Prepaid
...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.50,33.0,Prepaid
120,1091121305541,2001822466,10,1.376,9019,342301,Forward charges,121003,b,0.50,89.6,COD
121,1091121666133,2001823564,6,0.672,4803,492001,Forward and RTO charges,121003,d,1.25,86.7,Prepaid
122,1091121981575,2001825261,13,1.557,9049,517128,Forward and RTO charges,121003,d,1.25,176.3,Prepaid


**Calculate the COD charges. For this case, if the
order is Prepaid then COD charge is 0 and if the order is COD then we need to check
the amount of the order
If the order amount is less than equal to 300 then COD charge is 15
Else the COD charge is 5% of the Order Amount**

**Create Function called Condition1**

In [47]:
def condition1(s):
    if(s['Payment Mode']=='COD'):
        if(s['Item Price(Per Qty.)']<=300):
            return 15
        else:
            return s['Item Price(Per Qty.)']*0.05
    else:
        return 0

In [48]:
newdf2['COD']=newdf2.apply(condition1,axis=1)

In [49]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone),ChargesSlap,Payment Mode,COD
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.50,33.0,COD,66.20
1,1091117222065,2001806226,2,0.480,1125,723146,Forward charges,121003,d,1.25,45.4,COD,56.25
2,1091117222080,2001806229,3,0.500,1082,421204,Forward charges,121003,d,1.25,45.4,COD,54.10
3,1091117222124,2001806232,8,1.302,2139,507101,Forward charges,121003,d,1.25,90.2,COD,106.95
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.50,33.0,Prepaid,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.50,33.0,Prepaid,0.00
120,1091121305541,2001822466,10,1.376,9019,342301,Forward charges,121003,b,0.50,89.6,COD,450.95
121,1091121666133,2001823564,6,0.672,4803,492001,Forward and RTO charges,121003,d,1.25,86.7,Prepaid,0.00
122,1091121981575,2001825261,13,1.557,9049,517128,Forward and RTO charges,121003,d,1.25,176.3,Prepaid,0.00


***Calculating total charges by Company X***

In [50]:
newdf2['Total Charges']=newdf2['COD']+newdf2['ChargesSlap']

In [51]:
newdf2

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone),ChargesSlap,Payment Mode,COD,Total Charges
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.50,33.0,COD,66.20,99.20
1,1091117222065,2001806226,2,0.480,1125,723146,Forward charges,121003,d,1.25,45.4,COD,56.25,101.65
2,1091117222080,2001806229,3,0.500,1082,421204,Forward charges,121003,d,1.25,45.4,COD,54.10,99.50
3,1091117222124,2001806232,8,1.302,2139,507101,Forward charges,121003,d,1.25,90.2,COD,106.95,197.15
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.50,33.0,Prepaid,0.00,33.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.50,33.0,Prepaid,0.00,33.00
120,1091121305541,2001822466,10,1.376,9019,342301,Forward charges,121003,b,0.50,89.6,COD,450.95,540.55
121,1091121666133,2001823564,6,0.672,4803,492001,Forward and RTO charges,121003,d,1.25,86.7,Prepaid,0.00,86.70
122,1091121981575,2001825261,13,1.557,9049,517128,Forward and RTO charges,121003,d,1.25,176.3,Prepaid,0.00,176.30


In [52]:
t=newdf2

In [53]:
cour3=cour[['Order ID','Billing Amount (Rs.)']]

In [54]:
cour3

,Order ID,Billing Amount (Rs.)
0,2001806232,140.0
1,2001806273,101.2
2,2001806408,224.6
3,2001806458,61.3
4,2001807012,45.4
...,...,...
119,2001812941,95.2
120,2001809383,86.7
121,2001820978,45.4
122,2001811475,47.0


In [55]:
tt=pd.merge(left=t,right=cour3,on='Order ID')

In [56]:
tt

,AWB Code,Order ID,Order Qty,Total Weight,Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Applicable weight slab(based on Zone),ChargesSlap,Payment Mode,COD,Total Charges,Billing Amount (Rs.)
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.50,33.0,COD,66.20,99.20,174.5
1,1091117222065,2001806226,2,0.480,1125,723146,Forward charges,121003,d,1.25,45.4,COD,56.25,101.65,90.2
2,1091117222080,2001806229,3,0.500,1082,421204,Forward charges,121003,d,1.25,45.4,COD,54.10,99.50,90.2
3,1091117222124,2001806232,8,1.302,2139,507101,Forward charges,121003,d,1.25,90.2,COD,106.95,197.15,140.0
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.50,33.0,Prepaid,0.00,33.00,61.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.50,33.0,Prepaid,0.00,33.00,45.4
120,1091121305541,2001822466,10,1.376,9019,342301,Forward charges,121003,b,0.50,89.6,COD,450.95,540.55,135.0
121,1091121666133,2001823564,6,0.672,4803,492001,Forward and RTO charges,121003,d,1.25,86.7,Prepaid,0.00,86.70,172.8
122,1091121981575,2001825261,13,1.557,9049,517128,Forward and RTO charges,121003,d,1.25,176.3,Prepaid,0.00,176.30,345.0


**Create new column for calcuating Difference Between Expected Charges and Billed Charges (Rs.)**

In [57]:
tt['Difference Between Expected Charges and Billed Charges (Rs.)']=tt['Total Charges']-tt['Billing Amount (Rs.)']

In [58]:
math.ceil(1.376/0.50)

3

**Function called weightSlapX is create to calculate weight slap by Company X**

In [59]:
def weightSlapX(s):
    c=math.ceil(s['Total Weight']/s['Applicable weight slab(based on Zone)'])
    return c

**New column is create by using function weightSlapX**

In [60]:
tt['WeightslabX']=tt.apply(weightSlapX,axis=1)

**Renaming Column**

In [61]:
tt.rename(columns={'Total Weight':'TotalweightasperX(kg)','Zone':'DeliveryZoneasperX','Total Charges':'ExpectedChargeasperX(Rs.)','Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'},inplace=True)

In [62]:
tt

,AWB Code,Order ID,Order Qty,TotalweightasperX(kg),Item Price(Per Qty.),Customer Pincode,Type of Shipment,Warehouse Pincode,DeliveryZoneasperX,Applicable weight slab(based on Zone),ChargesSlap,Payment Mode,COD,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.),WeightslabX
0,1091117221940,2001806210,2,0.220,1324,140604,Forward charges,121003,b,0.50,33.0,COD,66.20,99.20,174.5,-75.30,1
1,1091117222065,2001806226,2,0.480,1125,723146,Forward charges,121003,d,1.25,45.4,COD,56.25,101.65,90.2,11.45,1
2,1091117222080,2001806229,3,0.500,1082,421204,Forward charges,121003,d,1.25,45.4,COD,54.10,99.50,90.2,9.30,1
3,1091117222124,2001806232,8,1.302,2139,507101,Forward charges,121003,d,1.25,90.2,COD,106.95,197.15,140.0,57.15,2
4,1091117222135,2001806233,2,0.245,1486,263139,Forward charges,121003,b,0.50,33.0,Prepaid,0.00,33.00,61.3,-28.30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,2,0.477,484,342008,Forward charges,121003,b,0.50,33.0,Prepaid,0.00,33.00,45.4,-12.40,1
120,1091121305541,2001822466,10,1.376,9019,342301,Forward charges,121003,b,0.50,89.6,COD,450.95,540.55,135.0,405.55,3
121,1091121666133,2001823564,6,0.672,4803,492001,Forward and RTO charges,121003,d,1.25,86.7,Prepaid,0.00,86.70,172.8,-86.10,1
122,1091121981575,2001825261,13,1.557,9049,517128,Forward and RTO charges,121003,d,1.25,176.3,Prepaid,0.00,176.30,345.0,-168.70,2


**Create new dataframe from tt dataframe by selecting columns required for Report**

In [63]:
finalt=tt[['Order ID','AWB Code','TotalweightasperX(kg)','WeightslabX','DeliveryZoneasperX','ExpectedChargeasperX(Rs.)',
           'Charges Billed by Courier Company (Rs.)','Difference Between Expected Charges and Billed Charges (Rs.)'
           ]]

**Now using courier company file**

In [64]:
cour

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0


In [65]:
courtest=cour

In [66]:
courtest

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0


**Getting weight slab by Courier Company**

In [67]:
courtest['Weight slab by Courier']=np.select([(newdf2['Zone']=='a'),(newdf2['Zone']=='b'),(newdf2['Zone']=='c'),
                                                           (newdf2['Zone']=='d'),(newdf2['Zone']=='e')
            ],[0.25,0.5,0.75,1.25,1.5])

In [68]:
courtest

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab by Courier
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0,0.50
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2,1.25
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,1.25
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.25
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.50
...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2,0.50
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.50
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,1.25
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0,1.25


In [69]:
def weightSlapX2(s):
    c=math.ceil(s['Charged Weight']/s['Weight slab by Courier'])
    return c

In [70]:
courtest['Weight slab charged by Courier Company (KG)']=courtest.apply(weightSlapX2,axis=1)

**Renaming some column of courtest file for report**

In [71]:
courtest.rename(columns={'Charged Weight':'Total weight as per Courier Company (KG)','Zone':'Delivery Zone charged by Courier Company'},inplace=True)

In [72]:
courtest1=courtest[['Order ID','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)','Delivery Zone charged by Courier Company']]

In [73]:
courtest1

,Order ID,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company
0,2001806232,1.30,3,d
1,2001806273,1.00,1,d
2,2001806408,2.50,2,d
3,2001806458,1.00,1,b
4,2001807012,0.15,1,d
...,...,...,...,...
119,2001812941,0.73,2,d
120,2001809383,0.50,1,d
121,2001820978,0.50,1,d
122,2001811475,0.50,1,b


In [74]:
final=pd.merge(left=finalt,right=courtest1,on='Order ID')

In [75]:
final

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,DeliveryZoneasperX,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company
0,2001806210,1091117221940,0.220,1,b,99.20,174.5,-75.30,2.92,6,b
1,2001806226,1091117222065,0.480,1,d,101.65,90.2,11.45,0.68,1,d
2,2001806229,1091117222080,0.500,1,d,99.50,90.2,9.30,0.71,1,d
3,2001806232,1091117222124,1.302,2,d,197.15,140.0,57.15,1.30,3,d
4,2001806233,1091117222135,0.245,1,b,33.00,61.3,-28.30,0.78,2,b
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1,b,33.00,45.4,-12.40,0.50,1,d
120,2001822466,1091121305541,1.376,3,b,540.55,135.0,405.55,1.10,1,d
121,2001823564,1091121666133,0.672,1,d,86.70,172.8,-86.10,0.70,2,d
122,2001825261,1091121981575,1.557,2,d,176.30,345.0,-168.70,1.60,2,d


In [76]:
final=final[['Order ID','AWB Code','TotalweightasperX(kg)','WeightslabX','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)','DeliveryZoneasperX','Delivery Zone charged by Courier Company','ExpectedChargeasperX(Rs.)',
           'Charges Billed by Courier Company (Rs.)','Difference Between Expected Charges and Billed Charges (Rs.)'
           ]]

In [77]:
final

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),DeliveryZoneasperX,Delivery Zone charged by Courier Company,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,1,2.92,6,b,b,99.20,174.5,-75.30
1,2001806226,1091117222065,0.480,1,0.68,1,d,d,101.65,90.2,11.45
2,2001806229,1091117222080,0.500,1,0.71,1,d,d,99.50,90.2,9.30
3,2001806232,1091117222124,1.302,2,1.30,3,d,d,197.15,140.0,57.15
4,2001806233,1091117222135,0.245,1,0.78,2,b,b,33.00,61.3,-28.30
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1,0.50,1,b,d,33.00,45.4,-12.40
120,2001822466,1091121305541,1.376,3,1.10,1,b,d,540.55,135.0,405.55
121,2001823564,1091121666133,0.672,1,0.70,2,d,d,86.70,172.8,-86.10
122,2001825261,1091121981575,1.557,2,1.60,2,d,d,176.30,345.0,-168.70


In [78]:
f=final

In [79]:
f

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),DeliveryZoneasperX,Delivery Zone charged by Courier Company,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,1,2.92,6,b,b,99.20,174.5,-75.30
1,2001806226,1091117222065,0.480,1,0.68,1,d,d,101.65,90.2,11.45
2,2001806229,1091117222080,0.500,1,0.71,1,d,d,99.50,90.2,9.30
3,2001806232,1091117222124,1.302,2,1.30,3,d,d,197.15,140.0,57.15
4,2001806233,1091117222135,0.245,1,0.78,2,b,b,33.00,61.3,-28.30
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1,0.50,1,b,d,33.00,45.4,-12.40
120,2001822466,1091121305541,1.376,3,1.10,1,b,d,540.55,135.0,405.55
121,2001823564,1091121666133,0.672,1,0.70,2,d,d,86.70,172.8,-86.10
122,2001825261,1091121981575,1.557,2,1.60,2,d,d,176.30,345.0,-168.70


**Calculating Correctly Charge, Overcharge, UnderCharge of Company X**

In [80]:
Correctly_Charges=f[f['ExpectedChargeasperX(Rs.)']==f['Charges Billed by Courier Company (Rs.)']]

In [81]:
Correctly_Charges

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),DeliveryZoneasperX,Delivery Zone charged by Courier Company,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
11,2001806458,1091117223244,0.700,2,1.00,1,b,b,61.3,61.3,0.0
31,2001807012,1091117229345,0.240,1,0.15,1,d,d,45.4,45.4,0.0
33,2001807058,1091117323112,1.168,3,1.15,3,b,b,89.6,89.6,0.0
35,2001807186,1091117323812,0.500,1,0.50,1,d,d,45.4,45.4,0.0
37,2001807290,1091117324206,0.500,1,0.50,1,d,d,45.4,45.4,0.0
44,2001807814,1091117326612,0.607,2,0.79,1,b,b,61.3,61.3,0.0
46,2001807930,1091117327695,0.240,1,0.15,1,d,d,45.4,45.4,0.0
48,2001807956,1091117327275,1.080,3,1.08,3,b,b,89.6,89.6,0.0
54,2001808118,1091117435134,0.343,1,0.50,1,b,b,33.0,33.0,0.0
59,2001808507,1091117436383,0.607,2,0.79,1,b,b,61.3,61.3,0.0


In [82]:
Correctly_Charges_Count=len(Correctly_Charges.index)

In [83]:
Correctly_Charges_Sum=Correctly_Charges['ExpectedChargeasperX(Rs.)'].sum()

In [84]:
Correctly_Charges_Count

10

In [85]:
Correctly_Charges_Sum

577.6999999999999

In [86]:
Overcharge=f[f['ExpectedChargeasperX(Rs.)']<f['Charges Billed by Courier Company (Rs.)']]

In [87]:
Overcharge

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),DeliveryZoneasperX,Delivery Zone charged by Courier Company,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,1,2.92,6,b,b,99.20,174.5,-75.30
4,2001806233,1091117222135,0.245,1,0.78,2,b,b,33.00,61.3,-28.30
5,2001806251,1091117222146,0.245,1,1.27,2,d,d,45.40,135.0,-89.60
7,2001806304,1091117222360,0.500,1,0.71,2,b,d,33.00,90.2,-57.20
8,2001806338,1091117222570,0.500,1,0.70,2,d,d,45.40,90.2,-44.80
...,...,...,...,...,...,...,...,...,...,...,...
117,2001821750,1091121034350,0.755,2,0.80,2,b,d,61.30,90.2,-28.90
118,2001821766,1091121034641,0.240,1,0.20,1,b,d,56.65,63.4,-6.75
119,2001821995,1091121183730,0.477,1,0.50,1,b,d,33.00,45.4,-12.40
121,2001823564,1091121666133,0.672,1,0.70,2,d,d,86.70,172.8,-86.10


In [88]:
Overcharge['Difference Between Expected Charges and Billed Charges (Rs.)']=Overcharge['Difference Between Expected Charges and Billed Charges (Rs.)'].abs()

In [89]:
Overcharge

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),DeliveryZoneasperX,Delivery Zone charged by Courier Company,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,1,2.92,6,b,b,99.20,174.5,75.30
4,2001806233,1091117222135,0.245,1,0.78,2,b,b,33.00,61.3,28.30
5,2001806251,1091117222146,0.245,1,1.27,2,d,d,45.40,135.0,89.60
7,2001806304,1091117222360,0.500,1,0.71,2,b,d,33.00,90.2,57.20
8,2001806338,1091117222570,0.500,1,0.70,2,d,d,45.40,90.2,44.80
...,...,...,...,...,...,...,...,...,...,...,...
117,2001821750,1091121034350,0.755,2,0.80,2,b,d,61.30,90.2,28.90
118,2001821766,1091121034641,0.240,1,0.20,1,b,d,56.65,63.4,6.75
119,2001821995,1091121183730,0.477,1,0.50,1,b,d,33.00,45.4,12.40
121,2001823564,1091121666133,0.672,1,0.70,2,d,d,86.70,172.8,86.10


In [90]:
Overcharge_Count=len(Overcharge.index)

In [91]:
Overcharge_Count

70

In [92]:
Overcharge_Sum=Overcharge['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

In [93]:
Overcharge_Sum

4178.700000000001

In [94]:
Undercharge=f[f['ExpectedChargeasperX(Rs.)']>f['Charges Billed by Courier Company (Rs.)']]

In [95]:
Undercharge

,Order ID,AWB Code,TotalweightasperX(kg),WeightslabX,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),DeliveryZoneasperX,Delivery Zone charged by Courier Company,ExpectedChargeasperX(Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
1,2001806226,1091117222065,0.480,1,0.68,1,d,d,101.65,90.2,11.45
2,2001806229,1091117222080,0.500,1,0.71,1,d,d,99.50,90.2,9.30
3,2001806232,1091117222124,1.302,2,1.30,3,d,d,197.15,140.0,57.15
6,2001806273,1091117222194,0.615,1,1.00,1,d,d,129.60,101.2,28.40
12,2001806471,1091117223351,1.621,4,1.70,4,b,d,330.05,179.8,150.25
15,2001806567,1091117224902,0.952,1,1.16,3,d,d,245.20,141.0,104.20
16,2001806575,1091117225016,0.500,1,0.68,2,b,b,87.10,71.3,15.80
17,2001806616,1091117225484,0.963,2,1.08,3,b,b,215.25,98.6,116.65
18,2001806652,1091117226221,0.500,1,0.69,2,d,d,99.50,90.2,9.30
19,2001806686,1091117229555,0.240,1,0.15,1,d,d,75.55,45.4,30.15


In [96]:
Undercharge_Count=len(Undercharge.index)

In [97]:
Undercharge_Count

44

In [98]:
Undercharge_Sum=Undercharge['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

In [99]:
Undercharge_Sum

3987.3

In [100]:
summary=['Count','Amount(Rs.)']

In [101]:
summary

['Count', 'Amount(Rs.)']

In [102]:
data=[['Total orders where X has been correctly charged',10,577.6999999999999],
      ['Total Orders where X has been overcharged',70,4178.700000000001],
      ['Total Orders where X has been undercharged',44,3987.3]]

In [103]:
Summary = pd.DataFrame(data, columns=['','Count','Amount(Rs.)'])

In [104]:
Summary

,,Count,Amount(Rs.)
0,Total orders where X has been correctly charged,10,577.7
1,Total Orders where X has been overcharged,70,4178.7
2,Total Orders where X has been undercharged,44,3987.3


In [105]:
Summary.to_excel('Output_data_2.xlsx')

In [106]:
final.to_excel('Output_Data_1.xlsx')